Awaitable-объяект используются с инструкцией await для переключения контекста

[Документация](https://docs.python.org/3/library/asyncio-task.html#awaitables)


In [1]:
import asyncio
import nest_asyncio
from functools import wraps

nest_asyncio.apply()

In [ ]:
import time


def func_timer(func):
    @wraps(func)
    async def wrapper(*args, **kwargs):
        start = time.time()
        result = await func(*args, **kwargs)
        end = time.time()
        print(f'📝 {func.__name__}: {end - start:.2f} секунд')
        return result

    return wrapper


## Coroutine

Объект, возвращаемый при вызове `async def`.\
Исполняется после `await`, запуска в `Task` или `gather`

В Цикле событий (ЦС):
1. При достижении `await` исполняемая корутина приостанавливает себя и ЦС "ждет" событие из корутины
2. В это время ЦС может заниматься другими корутинами
3. Если событие готово, ЦС возобновляет корутину
4. Результат возвращается на место await
5. Корутина идет до следующего await или до конца
6. Код, ожидавший корутину продолжается


In [4]:
async def async_func():
    await asyncio.sleep(1)
    return True


print(async_func.__class__)
coro = async_func()
print(coro.__class__)

await coro

<class 'function'>
<class 'coroutine'>


True

## Task (create_task())

1. Создается объект корутины
2. Корутина немедленно оборачивается в Task и регистрируется в ЦС
3. Когда ЦС получает управление, он запускает задачу и выполняет код корутины до await
4. Корутина останавливается (Задача в состоянии pending)
5. Событие готово:  ЦС возобновляет задачу, продолжая корутину до завершения.


In [ ]:
@func_timer
async def wake_up():
    print('🙋 Woked up')
    return True


@func_timer
async def make_sandwich():
    print('🚀 Start making sandwich...')
    await asyncio.sleep(1)
    return True


@func_timer
async def make_coffee():
    print('🚀 Start making coffee...')
    await asyncio.sleep(0.5)
    return True


@func_timer
async def go_to_work():
    await asyncio.sleep(3)
    return True


async def main():
    await wake_up()
    asyncio.create_task(make_sandwich())
    task_c = asyncio.create_task(make_coffee())
    print(f'{task_c.__dict__=}')
    print(task_c.get_coro())

    await asyncio.sleep(0)
    print('🏁 End of main...')


asyncio.run(main())

🙋 Woked up
📝 wake_up: 0.00 секунд
task_c.__dict__={'_loop': <_UnixSelectorEventLoop running=True closed=False debug=False>, '_callbacks': [], '_name': 'Task-13', '_num_cancels_requested': 0, '_must_cancel': False, '_fut_waiter': None, '_coro': <coroutine object make_coffee at 0x1043ffab0>, '_context': <_contextvars.Context object at 0x103d95780>}
<coroutine object make_coffee at 0x1043ffab0>
🚀 Start making sandwich...
🚀 Start making coffee...
🏁 End of main...


📝 make_coffee: 0.50 секунд
📝 make_sandwich: 1.00 секунд


### Состояния задачи

`done` - задача завершена
`cancelled` - задача отменена

In [ ]:
async def async_func():
    await asyncio.sleep(0.5)


task = asyncio.create_task(async_func())
print(task.done(), task.cancelled())

await task
print(task.done(), task.cancelled())

False False
True False


### Отмена задачи

In [21]:
task = asyncio.create_task(async_func())
task.cancel()

print(task.done(), task.cancelled())

try:
    await task
except asyncio.CancelledError:
    print('❌ Task was cancelled')

print(task.done(), task.cancelled())

False False
❌ Task was cancelled
True True


## Futures


Обычно используются на низком уровне для предоставления результатов async-операции. Редко создаются в коде.


In [6]:
async def afunc_with_future():
    f = asyncio.Future()
    await asyncio.sleep(0.5)
    f.set_result('✅ Future Result')
    return f

res = await afunc_with_future()
print(type(res))
print(await res)

<class 'asyncio.futures.Future'>
✅ Future Result


In [5]:
async def afunc():
    await asyncio.sleep(0.5)
    return '✅ Result'

f = asyncio.ensure_future(afunc())

type(f)

asyncio.tasks.Task